In [ ]:
# Install dependencies with pip (needed for Codespaces)
!pip install qiskit numpy matplotlib pylatexenc qiskit_ibm_runtime

In [ ]:
# Author: David Reese
# Simple Python RSA Encryption Algorithm

import base64
import hashlib
import time
import numpy
import random
import sys

# Increase the recursion limit for deep recursive functions
sys.setrecursionlimit(1500)

# Greatest Common Denominator
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a


# check if p is prime
# do the test k times
def fermat(p, k=2):
    # base case
    if p < 2:
        return False
    # quick check for small primes & common divisors
    if p in [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31]:
        return True
    if p % 2 == 0 or p % 3 == 0 or p % 5 == 0:
        return False
    # generate random number & test
    for _ in range(k):
        a = random.randint(2, p - 2)
        if pow(a, p - 1, p) != 1:
            return False
    # if the code reaches this point, unless it is a carmichael number,
    # we should have our result...
    return True


# divide-and-conquer algo for multiplying large numbers
# theoretically faster than FFT at O(n^1.58)
# STEPS:
# 1. split the input into two halves ( 7776 -> 77 and 76)
# 2. recursively multiply the low & high halves, and calculate product of sum of halves.
# 3. combine the final product.
def karatsuba(x, y):
    # base case:
    if x < 10 and y < 10:
        return x * y
    # first step
    len_x = len(str(x))
    len_y = len(str(y))
    # calculate midpoint m
    m = max(len_x, len_y) // 2
    a = x // 10 ** m
    b = x % 10 ** m
    c = y // 10 ** m
    d = y % 10 ** m
    # recursive steps
    ac = karatsuba(a, c)
    bd = karatsuba(b, d)
    product = karatsuba(a + b, c + d) - ac - bd
    # combine & return result
    return ac * 10 ** (2 * m) + (product * 10 ** m) + bd


# Generates e with a bias towards higher numbers
def generate_e(phi):
    # descending list of e candidates, likely 65537 will work
    candidates = [65537, 32767, 17011, 8191, 4093, 2039, 1021, 509, 257, 127, 61, 37, 17]
    for e in candidates:
        if gcd(e, phi) == 1:  # Is it co-prime?
            return e
    return None


def generate_rsa_keys():
    primes = []
    while len(primes) < 2:
        p = random.randint(2**15, 2**16)  # smaller primes for demonstration
        if fermat(p):
            primes.append(p)
    p, q = primes
    n = karatsuba(p, q)
    phi = karatsuba(p-1, q-1)
    e = generate_e(phi)
    d = pow(e, -1, phi)
    return {'p': p, 'q': q, 'n': n, 'phi': phi, 'e': e, 'd': d}


# Generate RSA keys
keys = generate_rsa_keys()
print("Generated RSA keys:", keys)